### Exercise 3 - Solution

Here we want to use Marshmallow to do the serialization and deserialization that we did in Exercises 1 and 2.

In [1]:
class Stock:
    def __init__(self, symbol, date, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        
class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume

In [2]:
from datetime import date, datetime
from decimal import Decimal

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22), 
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22), 
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

I'm first going to define some schemas for trades and stocks:

In [3]:
from marshmallow import Schema, fields

In [4]:
class StockSchema(Schema):
    symbol = fields.Str()
    date = fields.Date()
    open = fields.Decimal()
    high = fields.Decimal()
    low = fields.Decimal()
    close = fields.Decimal()
    volume = fields.Integer()

Let's test this one out quickly:

In [5]:
StockSchema().dump(Stock('TSLA', date(2018, 11, 22), 
                          Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), 
                          Decimal('338.19'), 365_607))

MarshalResult(data={'low': Decimal('337.60'), 'open': Decimal('338.19'), 'close': Decimal('338.19'), 'volume': 365607, 'symbol': 'TSLA', 'high': Decimal('338.64'), 'date': '2018-11-22'}, errors={})

That's great, but there's a slight issue - you'll notice that the marshalled data has `Decimal` objects for our prices. This is still going to be an issue if we try to serialize to JSON:

In [6]:
StockSchema().dumps(Stock('TSLA', date(2018, 11, 22), 
                          Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), 
                          Decimal('338.19'), 365_607))

TypeError: Object of type 'Decimal' is not JSON serializable

So let's fix that:

In [7]:
class StockSchema(Schema):
    symbol = fields.Str()
    date = fields.Date()
    open = fields.Decimal(as_string=True)
    high = fields.Decimal(as_string=True)
    low = fields.Decimal(as_string=True)
    close = fields.Decimal(as_string=True)
    volume = fields.Integer()

In [8]:
StockSchema().dump(Stock('TSLA', date(2018, 11, 22), 
                          Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), 
                          Decimal('338.19'), 365_607)).data

{'low': '337.60',
 'open': '338.19',
 'close': '338.19',
 'volume': 365607,
 'symbol': 'TSLA',
 'high': '338.64',
 'date': '2018-11-22'}

And now we can serialize to JSON:

In [9]:
StockSchema().dumps(Stock('TSLA', date(2018, 11, 22), 
                          Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), 
                          Decimal('338.19'), 365_607)).data

'{"low": "337.60", "open": "338.19", "close": "338.19", "volume": 365607, "symbol": "TSLA", "high": "338.64", "date": "2018-11-22"}'

Let's now handle the `Trade` schema:

In [10]:
class TradeSchema(Schema):
    symbol = fields.Str()
    timestamp = fields.DateTime()
    order = fields.Str()
    price = fields.Decimal(as_string=True)
    commission = fields.Decimal(as_string=True)
    volume = fields.Integer()

In [11]:
TradeSchema().dumps(Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99'))).data

'{"price": "338.25", "volume": 100, "symbol": "TSLA", "order": "buy", "commission": "9.99", "timestamp": "2018-11-22T10:05:12+00:00"}'

Now let's write a schema for our overall dictionary that contains a list of Trades and a list of Quotes:

In [12]:
class ActivitySchema(Schema):
    trades = fields.Nested(TradeSchema, many=True)
    quotes = fields.Nested(StockSchema, many=True)

And we can now serialize and deserialize:

In [13]:
result = ActivitySchema().dumps(activity, indent=2).data

In [14]:
type(result)

str

In [15]:
print(result)

{
  "trades": [
    {
      "price": "338.25",
      "volume": 100,
      "symbol": "TSLA",
      "order": "buy",
      "commission": "9.99",
      "timestamp": "2018-11-22T10:05:12+00:00"
    },
    {
      "price": "177.01",
      "volume": 20,
      "symbol": "AAPL",
      "order": "sell",
      "commission": "9.99",
      "timestamp": "2018-11-22T10:30:05+00:00"
    }
  ],
  "quotes": [
    {
      "low": "337.60",
      "open": "338.19",
      "close": "338.19",
      "volume": 365607,
      "symbol": "TSLA",
      "high": "338.64",
      "date": "2018-11-22"
    },
    {
      "low": "176.64",
      "open": "176.66",
      "close": "176.78",
      "volume": 3699184,
      "symbol": "AAPL",
      "high": "177.25",
      "date": "2018-11-22"
    },
    {
      "low": "103.07",
      "open": "103.25",
      "close": "103.11",
      "volume": 4493689,
      "symbol": "MSFT",
      "high": "103.48",
      "date": "2018-11-22"
    }
  ]
}


So a JSON string...
Let's deserialize that JSON string:

In [16]:
activity_deser = ActivitySchema().loads(result).data

In [17]:
type(activity_deser)

dict

In [18]:
from pprint import pprint

pprint(activity_deser)

{'quotes': [{'close': Decimal('338.19'),
             'date': datetime.date(2018, 11, 22),
             'high': Decimal('338.64'),
             'low': Decimal('337.60'),
             'open': Decimal('338.19'),
             'symbol': 'TSLA',
             'volume': 365607},
            {'close': Decimal('176.78'),
             'date': datetime.date(2018, 11, 22),
             'high': Decimal('177.25'),
             'low': Decimal('176.64'),
             'open': Decimal('176.66'),
             'symbol': 'AAPL',
             'volume': 3699184},
            {'close': Decimal('103.11'),
             'date': datetime.date(2018, 11, 22),
             'high': Decimal('103.48'),
             'low': Decimal('103.07'),
             'open': Decimal('103.25'),
             'symbol': 'MSFT',
             'volume': 4493689}],
 'trades': [{'commission': Decimal('9.99'),
             'order': 'buy',
             'price': Decimal('338.25'),
             'symbol': 'TSLA',
             'timestamp': datetim

That's looking pretty good, but you'll notice something - the objects in the `trades` and `quotes` list have been loaded into plain dictionary objects, not `Trade` and `Stock` objects:

In [19]:
type(activity_deser['trades'][0])

dict

For this we have to remember to provide functions decorated with `@post_load`:

In [20]:
from marshmallow import post_load

class TradeSchema(Schema):
    symbol = fields.Str()
    timestamp = fields.DateTime()
    order = fields.Str()
    price = fields.Decimal(as_string=True)
    commission = fields.Decimal(as_string=True)
    volume = fields.Integer()
    
    @post_load
    def make_trade(self, data):
        return Trade(**data)

In [21]:
class StockSchema(Schema):
    symbol = fields.Str()
    date = fields.Date()
    open = fields.Decimal(as_string=True)
    high = fields.Decimal(as_string=True)
    low = fields.Decimal(as_string=True)
    close = fields.Decimal(as_string=True)
    volume = fields.Integer()
    
    @post_load()
    def make_stock(self, data):
        return Stock(**data)

And of course we have to redefine our `ActivitySchema` to make sure it is referencing the newly defined sub schema classes:

In [22]:
class ActivitySchema(Schema):
    trades = fields.Nested(TradeSchema, many=True)
    quotes = fields.Nested(StockSchema, many=True)

And now we can try this again:

In [23]:
activity_deser = ActivitySchema().loads(result).data

TypeError: __init__() got an unexpected keyword argument 'open'

So here we have an issue - basically our method to construct a new `Stock` object expects the argument for the open price to be `open_`, and not `open` which is what our schema is producing.

We could do it in one of two ways:

First we can change our method that builds the `Stock` object:

In [24]:
class StockSchema(Schema):
    symbol = fields.Str()
    date = fields.Date()
    open = fields.Decimal(as_string=True)
    high = fields.Decimal(as_string=True)
    low = fields.Decimal(as_string=True)
    close = fields.Decimal(as_string=True)
    volume = fields.Integer()
    
    @post_load()
    def make_stock(self, data):
        data['open_'] = data.pop('open')
        return Stock(**data)

In [25]:
class ActivitySchema(Schema):
    trades = fields.Nested(TradeSchema, many=True)
    quotes = fields.Nested(StockSchema, many=True)

In [26]:
activity_deser = ActivitySchema().loads(result).data

In [27]:
pprint(activity_deser)

{'quotes': [<__main__.Stock object at 0x105e70e80>,
 'trades': [<__main__.Trade object at 0x105e70c18>,
            <__main__.Trade object at 0x105e70b70>]}


So, let's just recap the various schemas we have to create:

In [28]:
class StockSchema(Schema):
    symbol = fields.Str()
    date = fields.Date()
    open = fields.Decimal(as_string=True)
    high = fields.Decimal(as_string=True)
    low = fields.Decimal(as_string=True)
    close = fields.Decimal(as_string=True)
    volume = fields.Integer()
    
    @post_load()
    def make_stock(self, data):
        data['open_'] = data.pop('open')
        return Stock(**data)
    
class TradeSchema(Schema):
    symbol = fields.Str()
    timestamp = fields.DateTime()
    order = fields.Str()
    price = fields.Decimal(as_string=True)
    commission = fields.Decimal(as_string=True)
    volume = fields.Integer()
    
    @post_load
    def make_trade(self, data):
        return Trade(**data)
    
class ActivitySchema(Schema):
    trades = fields.Nested(TradeSchema, many=True)
    quotes = fields.Nested(StockSchema, many=True)

As you can see this is a whole lot easier than doing it by hand using the standard library.